# Entendimiento de los datos

A continuación, se inicia el proceso de comprensión de datos, cuyo objetivo principal es familiarizarse con la información, entender sus características internas y, posteriormente, iniciar el procesamiento adecuado para alimentar un modelo de red neuronal recurrente.

## Parte 1: Carga de librerías

Se procede a importar las bibliotecas que serán utilizadas a lo largo del desarrollo de este proyecto.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

## Parte 2: Entendimiento de los datos

Se procede con la carga de los datos con la ayuda de la libreria pandas.

In [3]:
df = pd.read_excel("base_datos/prueba.xlsx")
df.head(10)

,IdConsumibles,AnoMesTransferencia,Cantidad_transferida,Cod_cliente,Estado_transferencia
0,1,202112,97,624,Terminado
1,2,202112,29,843,Terminado
2,3,202112,99,864,Terminado
3,4,202112,33,598,Terminado
4,5,202112,20,607,Terminado
5,6,202112,19,700,Terminado
6,7,202112,20,571,Terminado
7,8,202112,90,578,Terminado
8,9,202201,55,805,Terminado
9,10,202201,53,794,Terminado


Identificar número de muestras y atributos

In [4]:
forma = df.shape
print("Cantidad de registros: {}".format(forma[0]))
print("Cantidad de columnas: {}".format(forma[1]))

Cantidad de registros: 1000
Cantidad de columnas: 5


Identificar el tipo de cada atributo

In [5]:
df.dtypes

IdConsumibles            int64
AnoMesTransferencia      int64
Cantidad_transferida     int64
Cod_cliente              int64
Estado_transferencia    object
dtype: object

**Nota:** Basándonos en el resultado anterior, se evidencia que únicamente una columna es categórica, mientras que las restantes son de naturaleza numérica.

Resumen medidas estadística básicas

In [6]:
df.describe()

,IdConsumibles,AnoMesTransferencia,Cantidad_transferida,Cod_cliente
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,202218.678000,51.155000,720.425000
std,288.819436,84.653296,28.702488,104.843753
min,1.000000,202106.000000,3.000000,536.000000
25%,250.750000,202112.000000,26.000000,627.000000
50%,500.500000,202206.000000,50.000000,724.500000
75%,750.250000,202304.000000,77.000000,811.250000
max,1000.000000,202402.000000,100.000000,896.000000


Valores únicos de las variables categóricas

In [7]:
df['Estado_transferencia'].unique()

array(['Terminado'], dtype=object)

**Nota:** Se observa que la columna 'Estado_transferencia' únicamente presenta la categoría 'Terminado'.

Identificar registros faltantes

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   IdConsumibles         1000 non-null   int64 
 1   AnoMesTransferencia   1000 non-null   int64 
 2   Cantidad_transferida  1000 non-null   int64 
 3   Cod_cliente           1000 non-null   int64 
 4   Estado_transferencia  1000 non-null   object
dtypes: int64(4), object(1)
memory usage: 39.2+ KB


**Nota:** Según la información proporcionada, no hay valores ausentes en el DataFrame. Esto se puede confirmar observando la columna "Non-Null Count" en el resumen de la información del DataFrame, que indica que todas las columnas tienen 1000 valores no nulos. Por lo tanto, todas las entradas tienen valores presentes en estas columnas y no hay valores faltantes.

Verificar si hay registros repeditos

In [9]:
duplicados = df.duplicated()
total_duplicados = duplicados.sum()
print("Cantidad registros duplicados: {}".format(total_duplicados))

Cantidad registros duplicados: 0


## Parte 3: Preparación de los datos

En la sección de preparación de datos, se llevarán a cabo las siguientes acciones:

1. Se eliminarán las columnas "idConsumibles" y "Estado_transferencia" del conjunto de datos. La columna "idConsumibles" consiste en un identificador único asignado a cada registro de la base de datos. Aunque útil para distinguir cada entrada individualmente, no añade información relevante para el análisis de pronóstico de demanda de consumibles. En cuanto a la columna "Estado_transferencia", indica el estado de la transferencia de consumibles, pero al mantenerse constante en todos los registros, no aporta variabilidad ni información significativa para el análisis. Adicionalmente, se eliminará la columna 'Cod_cliente' dado que no es relevante para los objetivos analíticos del ejercicio.

2. Se procederá a convertir las fechas al formato apropiado y a ordenarlas cronológicamente.

In [10]:
#Se crea una copa del dataframe original, para no ser afectado por la etapa de preparación
df_pro = df.copy()

### Eliminación de columnas

Se procede a eliminar las columnas "idConsumibles" y "Estado_transferencia".

In [11]:
df_pro.drop(columns=["IdConsumibles","Estado_transferencia", "Cod_cliente"], axis=1, inplace=True)
df_pro

,AnoMesTransferencia,Cantidad_transferida
0,202112,97
1,202112,29
2,202112,99
3,202112,33
4,202112,20
...,...,...
995,202209,12
996,202209,56
997,202209,11
998,202209,87


### Cambio formato de fechas
 


In [12]:
df_pro['AnoMesTransferencia'] = pd.to_datetime(df_pro['AnoMesTransferencia'], format='%Y%m')
df_pro.sort_values(by='AnoMesTransferencia', inplace=True)
df_pro.reset_index(drop=True, inplace=True)
df_pro.head()

,AnoMesTransferencia,Cantidad_transferida
0,2021-06-01,8
1,2021-06-01,38
2,2021-06-01,50
3,2021-06-01,28
4,2021-06-01,68


Con el objetivo de garantizar una estimación precisa de los insumos necesarios para cubrir la demanda proyectada en cada mes, se procede a organizar los datos por periodo mensual.

In [13]:
df_pro = df_pro.groupby(pd.Grouper(key='AnoMesTransferencia', freq='M')).sum().reset_index()
df_pro.head()

,AnoMesTransferencia,Cantidad_transferida
0,2021-06-30,1669
1,2021-07-31,2796
2,2021-08-31,1427
3,2021-09-30,1974
4,2021-10-31,1833


Se guarda el conjunto de datos procesado 

In [14]:
df_pro.to_excel("base_datos_procesada/datos_procesados.xlsx", index=False)

In [15]:
print(df_pro)

   AnoMesTransferencia  Cantidad_transferida
0           2021-06-30                  1669
1           2021-07-31                  2796
2           2021-08-31                  1427
3           2021-09-30                  1974
4           2021-10-31                  1833
5           2021-11-30                  1984
6           2021-12-31                  1836
7           2022-01-31                  2414
8           2022-02-28                  2103
9           2022-03-31                  2198
10          2022-04-30                  1898
11          2022-05-31                  2599
12          2022-06-30                  2002
13          2022-07-31                  2306
14          2022-08-31                  2015
15          2022-09-30                  1141
16          2022-10-31                   983
17          2022-11-30                   770
18          2022-12-31                  1008
19          2023-01-31                  1323
20          2023-02-28                  1057
21        

In [16]:
print(df)

     IdConsumibles  AnoMesTransferencia  Cantidad_transferida  Cod_cliente   
0                1               202112                    97          624  \
1                2               202112                    29          843   
2                3               202112                    99          864   
3                4               202112                    33          598   
4                5               202112                    20          607   
..             ...                  ...                   ...          ...   
995            996               202209                    12          572   
996            997               202209                    56          615   
997            998               202209                    11          763   
998            999               202209                    87          716   
999           1000               202209                    17          540   

    Estado_transferencia  
0              Terminado  
1        